# Lab 04_2: Two-qubit (CPW Resonator Bus Coupling)

### developed by Seong Hyeon Park (pajoheji0909@snu.ac.kr)

### Please find the attached homework at the end of this tutorial code

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from qiskit_metal import designs, MetalGUI, Dict

design = designs.DesignPlanar()
design.overwrite_enabled = True

In [ ]:
# set the overall layout size (X)
design.chips.main.size.size_x = '6mm'
# set the overall layout size (Y)
design.chips.main.size.size_y = '6mm'
# set the substrate thickness (Z)
design.chips.main.size.size_z = '-400um'

In [ ]:
# start GUI
gui = MetalGUI(design)

In [ ]:
hfss = design.renderers.hfss
# start ANSYS HFSS
hfss.start()

In [ ]:
import pyEPR as epr
import qiskit_metal as metal

## design two qubits capacitively coupled to a CPW resonator

In [ ]:
# for readout resonator (CPW)
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.launchpad_wb_driven import LaunchpadWirebondDriven

# there are various in-built qubit types in Qiskit-Metal, you should try it by yourself
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6

In [ ]:
cpw_w = '10 um'; cpw_g = '6 um'

tr1_Ljj = '13nH' ; tr1_Cjj = '2fF'
res1_l = '3.5 mm'; res1_coup_l = '600 um'; res1_coup_g = '2um'

tr2_Ljj = '13.5nH' ; tr2_Cjj = '2fF'
res2_l = '3.6 mm'; res2_coup_l = '600 um'; res2_coup_g = '2um'

busres_l = '8.5 mm'

design.delete_all_components()

launch_options = dict(
    chip='main', 
    pos_x="-2.12 mm", 
    pos_y="0mm",  
    orientation='360', 
    lead_length='0um',
    pad_width='300um',
    pad_height='300um',
    pad_gap='180um',
    taper_height='300um'
)
RP1 = LaunchpadWirebondDriven(design, 'RP1', options = launch_options)

launch_options = dict(
    chip='main', 
    pos_x="2.12 mm", 
    pos_y="0mm",  
    orientation='180', 
    lead_length='30um',
    pad_width='300um',
    pad_height='300um',
    pad_gap='180um',
    taper_height='300um'
)
RP2 = LaunchpadWirebondDriven(design, 'RP2', options = launch_options)

clt_options = dict(
    chip='main', 
    pos_x="-1mm", 
    pos_y="0mm",
    prime_width=cpw_w,
    prime_gap=cpw_g,
    second_width=cpw_w,
    second_gap=cpw_g,
    coupling_space=res1_coup_g,
    coupling_length=res1_coup_l,
    down_length='100um',
    fillet='40um',
    mirror=False,
    open_termination=False,
    hfss_wire_bonds = False
)
CLT1 = CoupledLineTee(design, 'CLT1', options = clt_options)

route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='RP1', pin='tie'),
                                     end_pin=Dict(component='CLT1',pin='prime_start')),
                     trace_width=cpw_w, trace_gap=cpw_g)
TL1 = RouteStraight(design, 'TL1', options=route_options)

clt_options = dict(
    chip='main', 
    pos_x="1mm", 
    pos_y="0mm",
    prime_width=cpw_w,
    prime_gap=cpw_g,
    second_width=cpw_w,
    second_gap=cpw_g,
    coupling_space=res1_coup_g,
    coupling_length=res1_coup_l,
    down_length='100um',
    fillet='40um',
    mirror=True,
    open_termination=False,
    hfss_wire_bonds = False
)
CLT2 = CoupledLineTee(design, 'CLT2', options = clt_options)

route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='CLT1', pin='prime_end'),
                                     end_pin=Dict(component='CLT2',pin='prime_start')),
                     trace_width=cpw_w, trace_gap=cpw_g)
TL2 = RouteStraight(design, 'TL2', options=route_options)

route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='CLT2', pin='prime_end'),
                                     end_pin=Dict(component='RP2',pin='tie')),
                     trace_width=cpw_w, trace_gap=cpw_g)
TL3 = RouteStraight(design, 'TL3', options=route_options)

tr_options = dict(pos_x="-0.9mm", pos_y="-1.7mm", orientation = 180,
                  pad_width = '600um', pad_height = '60um', pad_gap = '30um',
                  pocket_height = '300um', pocket_width = '700um', inductor_width = '10um',
                  connection_pads=dict(readout = dict(loc_W=0,loc_H=-1, 
                                                      pad_height='20um', pad_width='200um', pad_gap='20um', pad_shift='0um',
                                                      cpw_width=cpw_w, cpw_gap=cpw_g, cpw_extend='-120um',pad_cpw_extent='-120um'),
                                       qubus1 = dict(loc_W=-1,loc_H=+1, 
                                                     pad_height='30um', pad_width='150um', pad_gap='10um', pad_cpw_shift='10um', pad_shift='0um',
                                                     cpw_width=cpw_w, cpw_gap=cpw_g, cpw_extend='0 um')),
                  hfss_inductance = tr1_Ljj,
                  hfss_capacitance = tr1_Cjj
                 )
Q1 = TransmonPocket6(design, 'Q1', options = tr_options)

route_options = Dict(chip='main',
                     hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='Q1',pin='readout'),
                                     end_pin=Dict(component='CLT1',pin='second_end')),
                     trace_width=cpw_w,
                     trace_gap=cpw_g,
                     total_length=res1_l,
                     fillet="80um",
                     meander = dict(spacing = '300um', asymmetry = '300um'),
                     lead = dict(start_straight = '100um', end_straight = '100 um'))
RR1 = RouteMeander(design, 'RR1', options=route_options)

qctrl1open = OpenToGround(design, 'QCTRL1OPEN', options=dict(pos_x='-0.9mm',  pos_y='-1.95mm', orientation='90'))

launch_options = dict(
    chip='main', 
    pos_x="-0.9mm", 
    pos_y="-2.38mm",  
    orientation='90', 
    lead_length='0um',
    pad_width='200um',
    pad_height='200um',
    pad_gap='120um',
    taper_height='200um'
)
QCTRLP1 = LaunchpadWirebondDriven(design, 'QCTRLP1', options = launch_options)

route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='QCTRL1OPEN', pin='open'),
                                     end_pin=Dict(component='QCTRLP1',pin='tie')),
                     trace_width=cpw_w, trace_gap=cpw_g)
QCTRLR1 = RouteStraight(design, 'QCTRLR1', options=route_options)

tr_options = dict(pos_x="0.9mm", pos_y="-1.7mm", orientation = 180,
                  pad_width = '600um', pad_height = '60um', pad_gap = '30um',
                  pocket_height = '300um', pocket_width = '700um', inductor_width = '10um',
                  connection_pads=dict(readout = dict(loc_W=0,loc_H=-1, 
                                                      pad_height='20um', pad_width='200um', pad_gap='20um', pad_shift='0um',
                                                      cpw_width=cpw_w, cpw_gap=cpw_g, cpw_extend='-120um',pad_cpw_extent='-120um'),
                                       qubus2 = dict(loc_W=1,loc_H=+1, 
                                                     pad_height='30um', pad_width='150um', pad_gap='10um', pad_cpw_shift='10um', pad_shift='0um',
                                                     cpw_width=cpw_w, cpw_gap=cpw_g, cpw_extend='0 um')),
                  hfss_inductance = tr2_Ljj,
                  hfss_capacitance = tr2_Cjj
                 )
Q2 = TransmonPocket6(design, 'Q2', options = tr_options)

route_options = Dict(chip='main',
                     hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='Q2',pin='readout'),
                                     end_pin=Dict(component='CLT2',pin='second_end')),
                     trace_width=cpw_w,
                     trace_gap=cpw_g,
                     total_length=res2_l,
                     fillet="80um",
                     meander = dict(spacing = '210um', asymmetry = '-300um'),
                     lead = dict(start_straight = '100um', end_straight = '100 um'))
RR2 = RouteMeander(design, 'RR2', options=route_options)

launch_options = dict(
    chip='main', 
    pos_x="0.9mm", 
    pos_y="-2.38mm",  
    orientation='90', 
    lead_length='0um',
    pad_width='200um',
    pad_height='200um',
    pad_gap='120um',
    taper_height='200um'
)
QCTRLP2 = LaunchpadWirebondDriven(design, 'QCTRLP2', options = launch_options)

qctrl2open = OpenToGround(design, 'QCTRL2OPEN', options=dict(pos_x='0.9mm',  pos_y='-1.95mm', orientation='90'))

route_options = Dict(hfss_wire_bonds = True,
                     pin_inputs=Dict(start_pin=Dict(component='QCTRL2OPEN', pin='open'),
                                     end_pin=Dict(component='QCTRLP2',pin='tie')),
                     trace_width=cpw_w, trace_gap=cpw_g)
QCTRLR2 = RouteStraight(design, 'QCTRLR2', options=route_options)

route_options = Dict(hfss_wire_bonds = False,
                     pin_inputs=Dict(start_pin=Dict(component='Q1', pin='qubus1'),
                                     end_pin=Dict(component='Q2',pin='qubus2')),
                     trace_width=cpw_w,
                     trace_gap=cpw_g,
                     total_length=busres_l,
                     fillet="50um",
                     meander = dict(spacing = '120um', asymmetry = '0um'),
                     lead = dict(start_straight = '100um', end_straight = '10 um'))
QQBUS1 = RouteMeander(design, 'QQBUS1', options=route_options)

gui.rebuild()
gui.autoscale()

# take a screenshot of the GUI
gui.screenshot()

## draw selected components only for simplicity
* Transmon qubits and coupler bus

In [ ]:
cpw_w = '10 um'; cpw_g = '6 um'

tr1_Ljj = '13nH' ; tr1_Cjj = '2fF'
res1_l = '3.5 mm'; res1_coup_l = '600 um'; res1_coup_g = '2um'

tr2_Ljj = '13.5nH' ; tr2_Cjj = '2fF'
res2_l = '3.6 mm'; res2_coup_l = '600 um'; res2_coup_g = '2um'

busres_l = '8.5 mm'

design.delete_all_components()

tr_options = dict(pos_x="-0.9mm", pos_y="-1.7mm", orientation = 180,
                  pad_width = '600um', pad_height = '60um', pad_gap = '30um',
                  pocket_height = '300um', pocket_width = '700um', inductor_width = '10um',
                  connection_pads=dict(qubus1 = dict(loc_W=-1,loc_H=+1, 
                                                     pad_height='30um', pad_width='150um', pad_gap='10um', pad_cpw_shift='10um', pad_shift='0um',
                                                     cpw_width=cpw_w, cpw_gap=cpw_g, cpw_extend='0 um')),
                  hfss_inductance = tr1_Ljj,
                  hfss_capacitance = tr1_Cjj
                 )
Q1 = TransmonPocket6(design, 'Q1', options = tr_options)

tr_options = dict(pos_x="0.9mm", pos_y="-1.7mm", orientation = 180,
                  pad_width = '600um', pad_height = '60um', pad_gap = '30um',
                  pocket_height = '300um', pocket_width = '700um', inductor_width = '10um',
                  connection_pads=dict(qubus2 = dict(loc_W=1,loc_H=+1, 
                                                     pad_height='30um', pad_width='150um', pad_gap='10um', pad_cpw_shift='10um', pad_shift='0um',
                                                     cpw_width=cpw_w, cpw_gap=cpw_g, cpw_extend='0 um')),
                  hfss_inductance = tr2_Ljj,
                  hfss_capacitance = tr2_Cjj
                 )
Q2 = TransmonPocket6(design, 'Q2', options = tr_options)

route_options = Dict(hfss_wire_bonds = False,
                     pin_inputs=Dict(start_pin=Dict(component='Q1', pin='qubus1'),
                                     end_pin=Dict(component='Q2',pin='qubus2')),
                     trace_width=cpw_w,
                     trace_gap=cpw_g,
                     total_length=busres_l,
                     fillet="50um",
                     meander = dict(spacing = '120um', asymmetry = '0um'),
                     lead = dict(start_straight = '100um', end_straight = '10 um'))
QQBUS1 = RouteMeander(design, 'QQBUS1', options=route_options)

gui.rebuild()
gui.autoscale()

# take a screenshot of the GUI
gui.screenshot()

# method (1): Energy Participation Ratio (EPR)

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis

# project name
proj_name = "lab04_two_qubits_cpwbus"

eig_s = EPRanalysis(design, "hfss")
hfss = eig_s.sim.renderer

try:
    hfss.start()
except:
    hfss.activate_design(proj_name)

In [ ]:
from qiskit_metal.analyses.quantization import EPRanalysis

# define the EPR analysis class
eig_s = EPRanalysis(design, "hfss")

# define the renderer program for the eigenmode simulation
hfss = eig_s.sim.renderer

# try-except to avoid common(?) error during the HFSS rendering
try:
    hfss.start()
except:
    hfss.activate_design(proj_name)

# NOTE: you should define the junction in HFSS
eig_s.setup.junctions.jj1.rect = 'JJ_rect_Lj_Q1_rect_jj'
eig_s.setup.junctions.jj1.line = 'JJ_Lj_Q1_rect_jj_'

eig_s.setup.junctions.jj2.rect = 'JJ_rect_Lj_Q2_rect_jj'
eig_s.setup.junctions.jj2.line = 'JJ_Lj_Q2_rect_jj_'

# define the setup
em_s = eig_s.sim.setup
em_s.name = "setup_custom"

# minimum searching frequency
em_s.min_freq = 3

# target eigenmode number
# NOTE: now you have two components... you should find 'three' eigenmodes (two for qubits and one for CPW bus)
em_s.n_modes = 3

# maximum pass number
em_s.max_passes = 15

# criteria for eigenmode simulation convergence: maximum delta F_eigen between passes < value
em_s.max_delta_f = 0.5

# simulation minimum converged passes 
em_s.min_converged = 2

# simulation order 0: point, 1: line, 2: face
em_s.basis_order = 1

# Design variables can also be added in for direct simulation sweeps. Here, we have to set the junction inductance and capacitance
em_s.vars = Dict({'Lj1': tr1_Ljj, 'Cj1': tr1_Cjj,
                  'Lj2': tr2_Ljj, 'Cj2': tr2_Cjj})


In [ ]:
eig_s.sim._render(name=proj_name,                # design name
                  selection = [],                # if None --> all
                  solution_type='eigenmode',     # NOTE: case sensitive
                  vars_to_initialize=em_s.vars,  # set variables
                  open_pins=[],                  # set open pins
                  port_list=[],                  # set ports, for eigenmode simulations --> R = 50 Ohm boundary conditions 
                  box_plus_buffer = True)        # if False --> set the overall layout as you defined above. If True --> set marginal chip sizes as you set here.

In [ ]:
# mesh setting name, components, maximum mesh length
hfss.modeler.mesh_length('qubitpad_mesh_setting', 
                         ['pad_bot_Q1', 'pad_top_Q1',
                          'pad_bot_Q2', 'pad_top_Q2'], 
                         MaxLength='0.1 mm')

hfss.modeler.mesh_length('cpw_mesh_setting', 
                         ['trace_QQBUS1', 'qubus1_connector_pad_Q1', 'qubus1_wire_Q1', 'qubus2_connector_pad_Q2', 'qubus2_wire_Q2'], 
                         MaxLength='0.05 mm')

In [ ]:
eig_s.sim._analyze()

In [ ]:
pinfo = hfss.pinfo
pinfo.junctions['jj1'] = {'rect':'JJ_rect_Lj_Q1_rect_jj', 'line':'JJ_Lj_Q1_rect_jj_',
                         'Lj_variable':'Lj1', 'Cj_variable':'Cj1'}

pinfo.junctions['jj2'] = {'rect':'JJ_rect_Lj_Q2_rect_jj', 'line':'JJ_Lj_Q2_rect_jj_',
                         'Lj_variable':'Lj2', 'Cj_variable':'Cj2'}

pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied

In [ ]:
eprd = epr.DistributedAnalysis(pinfo)
eprd.do_EPR_analysis()

In [ ]:
epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 8, fock_trunc = 10)

# Method (2): Capacitance Analysis and LOM derivation using the analysis package

In [ ]:
import scqubits as scq

from scipy.constants import speed_of_light as c_light
from qiskit_metal.analyses.quantization import LOManalysis
from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem, QuantumSystemRegistry
c1 = LOManalysis(design, "q3d")

In [ ]:
# example: update single setting
c1.sim.setup.max_passes = 30
c1.sim.setup.freq_ghz = 5

# example: update multiple settings
c1.sim.setup_update(solution_order = 'Medium', auto_increase_solution_order = 'False')

c1.sim.setup

In [ ]:
# you should define the open terminations, where coupling structures of a qubit end
c1.sim.run(components=['Q1'], open_terminations=[('Q1', 'qubus1')])

# c1.setup.run <- direct access
c1.sim.print_run_args()

q1_cmatrix = c1.sim.capacitance_matrix

# you should define the open terminations, where coupling structures of a qubit end
c1.sim.run(components=['Q2'], open_terminations=[('Q2', 'qubus2')])

# c1.setup.run <- direct access
c1.sim.print_run_args()

q2_cmatrix = c1.sim.capacitance_matrix

In [ ]:
# cell 1: transmon 1

opt1 = dict(cap_mat = q1_cmatrix,
            ind_dict = {('pad_top_Q1', 'pad_bot_Q1'):13},  # junction inductance in nH
            jj_dict = {('pad_top_Q1', 'pad_bot_Q1'):'j1'},
            cj_dict = {('pad_top_Q1', 'pad_bot_Q1'):2},    # junction capacitance in fF
            )
cell_1 = Cell(opt1)

# cell 2: transmon 2
opt2 = dict(cap_mat = q2_cmatrix,
            ind_dict = {('pad_top_Q2', 'pad_bot_Q2'): 13.5}, # junction inductance in nH
            jj_dict = {('pad_top_Q2', 'pad_bot_Q2'):'j2'},
            cj_dict = {('pad_top_Q2', 'pad_bot_Q2'):2},      # junction capacitance in fF
            )
cell_2 = Cell(opt2)

In [ ]:
# subsystem 1: transmon qubit Q1
transmon_Q1 = Subsystem(name='Q1', sys_type='TRANSMON', nodes=['j1'])

# subsystem 2: transmon qubit Q2
transmon_Q2 = Subsystem(name='Q2', sys_type='TRANSMON', nodes=['j2'])

# subsystem 3: bus resonator RR1
q_opts = dict(f_res = 6.70,           # bus dressed frequency in GHz
              Z0 = 50,                # characteristic impedance in Ohm
              vp = 0.404314 * c_light # phase velocity 
              )
res_BUS = Subsystem(name='res_BUS', sys_type='TL_RESONATOR', nodes=['qubus1_connector_pad_Q1', 'qubus2_connector_pad_Q2'], q_opts=q_opts)

In [ ]:
composite_sys = CompositeSystem(subsystems=[transmon_Q1, transmon_Q2, res_BUS], 
                                cells=[cell_1, cell_2], 
                                grd_node='ground_main_plane',
                                nodes_force_keep=['qubus1_connector_pad_Q1', 'qubus2_connector_pad_Q2']
                                )

In [ ]:
cg = composite_sys.circuitGraph()
print(cg)

In [ ]:
hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

In [ ]:
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

In [ ]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)

In [ ]:
composite_sys.compute_gs()

## Analytic expressions to calculate $\nu_{ZZ}$ between qubits

The full Hamiltonian to model the two-qubit, capacitively coupled to a bus resonator, can be expressed as follows

$\mathcal{H}=\sum_{q=1}^{2}[\omega_q a^\dagger_q a_q + \alpha_q a^\dagger_q a_q(a^\dagger_q a_q-1)]+\omega_r c^\dagger c + \sum_{q=1}^{2}g_{q,r}(a^\dagger_qc+a_qc^\dagger),$

where $\omega_q, \omega_r, g_{q,r}$ are the qubit frequency, bus resonator frequency, and coupling strength between a qubit and a bus resonator.
Note that $a, c$ are the annihilation operators and $a^\dagger, c^\dagger$ are creation operators, respectively.

The effective coupling ($J$) between qubits can be derived as 

$J = \frac{g_{1r}g_{2r}(\omega_1+\omega_2-2\omega_r)}{2(\omega_1-\omega_r)(\omega_2-\omega_r)}$,

and the ZZ coupling ($\nu_{ZZ}$) between qubits can be calculated as 

$\nu_{ZZ}=-\frac{2J^2(\alpha_1+\alpha_2)}{(\Delta_{12}+\alpha_1)(\Delta_{12}+\alpha_2)},$

where $\Delta_{12}=\omega_{1}-\omega_2$ is detuning frequency between qubits.

Reference: E Magesan and JM Gambetta, "Effective Hamiltonian models of the crossresonance gate," *Phys. Rev. A,* **101**, 052308 (2020) DOI: https://doi.org/10.1103/PhysRevA.101.052308

In [ ]:
from qutip import qeye, destroy, tensor

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# circuit parameters, unit: Hz
fQ1 = 4.855*1e9
fQ2 = 4.759*1e9
fbus = 6.704*1e9

alpha1 = -301.7*1e6
alpha2 = -302.5*1e6
g1 = 67.3*1e6
g2 = 67.3*1e6

In [ ]:
# detuning frequency
delta12 = fQ1 - fQ2

# effective coupling
J = g1*g2*(fQ1+fQ2-2*fbus)/(2*(fQ1-fbus)*(fQ2-fbus))

nu_ZZ = -2*J**2*(alpha1+alpha2)/((delta12+alpha1)*(delta12+alpha2))

print(f'The effective coupling J: {J/1e6:.3F} MHz and the ZZ coupling: {nu_ZZ/1e3:.3F} kHz')

From the definition of the ZZ coupling ($\nu_{ZZ}$), we can calculate $\nu_{ZZ}$ by numerically diagonalizing the Hamiltonian as 

$\nu_{ZZ} = \tilde{\omega}_{11} + \tilde{\omega}_{00} - \tilde{\omega}_{10} - \tilde{\omega}_{01},$

where $\tilde{\omega}_{ij}$ is the dressed state frequency (eigenfrequency) when qubit 1 is in $i$ state and qubit 2 is in $j$ state.

In [ ]:
# circuit parameters, unit: Hz
fQ1 = 4.855*1e9
fQ2 = 4.759*1e9
fbus = 6.704*1e9

alpha1 = -301.7*1e6
alpha2 = -302.5*1e6
g1 = 67.3*1e6
g2 = 67.3*1e6

In [ ]:
# Hamiltonian with h=1 assumption
# order number
N = 3

# qubit annihilation oeprators
a1 = tensor(destroy(N), qeye(N), qeye(N))
a2 = tensor(qeye(N), destroy(N), qeye(N))
# resonator annihilation operators
br = tensor(qeye(N), qeye(N), destroy(N))

# Hamiltonian
H0 = (fQ1*a1.dag()*a1 + alpha1/2*a1.dag()*a1*(a1.dag()*a1-1) +
      fQ2*a2.dag()*a2 + alpha2/2*a2.dag()*a2*(a2.dag()*a2-1) +
      fbus*br.dag()*br + 
      g1*(br*a1.dag()+br.dag()*a1) + g2*(br*a2.dag()+br.dag()*a2))

# unitary diagonalization and dressed state frequency
H0diag = np.sort(np.linalg.eigh(H0)[0])
eigfreqs = []
for j in range(len(H0diag)):
      eigfreqs.append(H0diag[j]-H0diag[0])

# f_110 - f_010 - f_100 + f_000 
with np.printoptions(precision=3, suppress=True):
      print(f'H_diag: {np.array(eigfreqs)},\n\nVzz (numerical): {(eigfreqs[6]+eigfreqs[0]-eigfreqs[1]-eigfreqs[2])/1e3:.3F} kHz')

# LAB HOMEWORK

### Directly coupled transmon qubit system simulations. You should achieve the target parameters within 5\%. The overall circuit layout, electric field distributions, simulations settings in ANSYS (including $L_J, C_J$ values), and the convergence plot vs. adaptive pass should be attached to the report.

1. Sweep $L_{J1}$ and $L_{J2}$ for two qubits. Calculate $\omega_{q1}/2\pi, \omega_{q2}/2\pi$, and $\chi_{12}/2\pi$. If possible, compare the results using the analytic equations.
2. Sweep the length of a bus resonator. Calculate $\omega_{q1}/2\pi, \omega_{q2}/2\pi, \omega_{\mathrm{bus}}/2\pi$, and $\chi_{12}/2\pi$. If possible, compare the results using the analytic equations.
2. Design two qubits, coupled via a CPW resonator bus, to have $\omega_{q1}/2\pi=4.5$ GHz, $\omega_{q2}/2\pi=5$ GHz, and $\chi_{12}/2\pi<30$ kHz from the `EPR` method. You have to modify the geometric features of a transmon qubit, a bus resonator, and a Josephson junction.
3. Design two qubits, coupled via a CPW resonator bus, to have $\omega_{q1}/2\pi=4.5$ GHz, $\omega_{q2}/2\pi=5$ GHz, and $\chi_{12}/2\pi<30$ kHz from the `new LOM` method. You have to modify the geometric features of a transmon qubit, a bus resonator, and a Josephson junction.